<a href="https://colab.research.google.com/github/titan-spyer/Credit_score/blob/main/Credit_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import kagglehub
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
path = kagglehub.dataset_download("lihxlhx/give-me-some-credit")

print("Path to dataset files:", path)
print(os.listdir(path))

100%|██████████| 5.16M/5.16M [00:00<00:00, 7.19MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/lihxlhx/give-me-some-credit/versions/1
['cs-training.csv', 'sampleEntry.csv', 'Data Dictionary.xls', 'cs-test.csv']


In [6]:
df = pd.read_csv(os.path.join(path, "cs-training.csv"))
df = df.drop("Unnamed: 0", axis=1)
df.isnull().sum()

,0
SeriousDlqin2yrs,0
RevolvingUtilizationOfUnsecuredLines,0
age,0
NumberOfTime30-59DaysPastDueNotWorse,0
DebtRatio,0
MonthlyIncome,29731
NumberOfOpenCreditLinesAndLoans,0
NumberOfTimes90DaysLate,0
NumberRealEstateLoansOrLines,0
NumberOfTime60-89DaysPastDueNotWorse,0


In [8]:
df["MonthlyIncome"].fillna(df["MonthlyIncome"].median(), inplace=True)
df["NumberOfDependents"].fillna(df["NumberOfDependents"].median(), inplace=True)
df["NumberOfTimes90DaysLate"] = df["NumberOfTimes90DaysLate"].clip(0, 10)
df["NumberOfTime30-59DaysPastDueNotWorse"] = df["NumberOfTime30-59DaysPastDueNotWorse"].clip(0, 10)
df["NumberOfTime60-89DaysPastDueNotWorse"] = df["NumberOfTime60-89DaysPastDueNotWorse"].clip(0, 10)


/tmp/ipython-input-2185220177.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["MonthlyIncome"].fillna(df["MonthlyIncome"].median(), inplace=True)
/tmp/ipython-input-2185220177.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

In [9]:
df["IncomeToDebt"] = df["MonthlyIncome"] / (df["DebtRatio"] + 1)
df["LatePaymentScore"] = (
    df["NumberOfTimes90DaysLate"] * 3 +
    df["NumberOfTime60-89DaysPastDueNotWorse"] * 2 +
    df["NumberOfTime30-59DaysPastDueNotWorse"]
)
df["HighRisk"] = (df["LatePaymentScore"] >= 5).astype(int)


In [11]:
X = df.drop("SeriousDlqin2yrs", axis=1)
y = df["SeriousDlqin2yrs"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [12]:
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    class_weight="balanced",
    random_state=42
)
rf.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=300,
                       random_state=42)

In [14]:
from sklearn.metrics import roc_auc_score
pred = rf.predict(X_test)
prob = rf.predict_proba(X_test)[:, 1]

print(classification_report(y_test, pred))
print("ROC-AUC:", roc_auc_score(y_test, prob))

              precision    recall  f1-score   support

           0       0.98      0.84      0.90     27995
           1       0.24      0.73      0.36      2005

    accuracy                           0.83     30000
   macro avg       0.61      0.78      0.63     30000
weighted avg       0.93      0.83      0.86     30000

ROC-AUC: 0.8647142369117393


In [16]:
joblib.dump(rf, "credit_model.pkl")
joblib.dump(scaler, "credit_scaler.pkl")
print("Saved credit_model.pkl and credit_scaler.pkl")

Saved credit_model.pkl and credit_scaler.pkl
